In [27]:
import datetime
import random
from datetime import datetime, timedelta
import json
from pymongo import MongoClient
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

print("All packages are working correctly!")



All packages are working correctly!


In [24]:
# 

# ----------------------------------------------------
# 1) Configuration de la connexion Mongo et SEEDING
# ----------------------------------------------------

In [23]:

MONGO_URI = "mongodb://localhost:27017/"   
client = MongoClient(MONGO_URI)
db = client["movie_recommender_db"]

app = Flask(__name__)
 
def seed_data():
    # Suppression des données existantes
    db.users.delete_many({})
    db.movies.delete_many({})
 
    moroccan_movies = [
        {"_id": "movie_ma_001", "title": "Ali Zaoua", "genres": ["Drame"], "actors": ["Said Taghmaoui", "Mounim Kbab"], "description": "Un regard poignant sur les enfants des rues à Casablanca.", "release_date": "2000-10-04"},
        {"_id": "movie_ma_002", "title": "Les Chevaux de Dieu", "genres": ["Drame", "Thriller"], "actors": ["Abdelhakim Rachid", "Abdelilah Rachid"], "description": "L'histoire de jeunes hommes dans un quartier défavorisé.", "release_date": "2012-05-23"},
        {"_id": "movie_ma_003", "title": "Much Loved", "genres": ["Drame"], "actors": ["Loubna Abidar", "Asmaa Lazrak"], "description": "Un portrait réaliste de la vie des travailleuses du sexe au Maroc.", "release_date": "2015-05-19"},
        {"_id": "movie_ma_004", "title": "Casablanca Beats", "genres": ["Musique", "Drame"], "actors": ["Ismail Adouab", "Nouhaila Arif"], "description": "Un atelier de hip-hop qui inspire les jeunes à s'exprimer.", "release_date": "2021-07-14"},
        {"_id": "movie_ma_005", "title": "Road to Kabul", "genres": ["Comédie", "Action"], "actors": ["Adil Abatourab", "Younes Bouab"], "description": "Quatre amis embarquent dans une aventure inattendue.", "release_date": "2012-07-11"}
    ]
 
    top_movies = [
        {"_id": "movie_int_001", "title": "The Godfather", "genres": ["Crime", "Drama"], "actors": ["Marlon Brando", "Al Pacino"], "description": "L'ascension et la chute de la famille Corleone.", "release_date": "1972-03-24"},
        {"_id": "movie_int_002", "title": "The Shawshank Redemption", "genres": ["Drama"], "actors": ["Tim Robbins", "Morgan Freeman"], "description": "L'histoire d'amitié et d'espoir entre deux prisonniers.", "release_date": "1994-09-23"},
        {"_id": "movie_int_003", "title": "The Dark Knight", "genres": ["Action", "Crime", "Drama"], "actors": ["Christian Bale", "Heath Ledger"], "description": "Batman affronte le Joker dans une bataille pour Gotham.", "release_date": "2008-07-18"},
        {"_id": "movie_int_004", "title": "Pulp Fiction", "genres": ["Crime", "Drama"], "actors": ["John Travolta", "Uma Thurman"], "description": "Des histoires interconnectées dans le monde du crime à Los Angeles.", "release_date": "1994-10-14"},
        {"_id": "movie_int_005", "title": "Schindler's List", "genres": ["Biography", "Drama", "History"], "actors": ["Liam Neeson", "Ralph Fiennes"], "description": "Un homme d'affaires sauve des centaines de juifs pendant la Seconde Guerre mondiale.", "release_date": "1993-12-15"},
        {"_id": "movie_int_006", "title": "Fight Club", "genres": ["Drama"], "actors": ["Brad Pitt", "Edward Norton"], "description": "Un homme découvre un mode de vie alternatif grâce à un club clandestin.", "release_date": "1999-10-15"}
    ]

    # Insertion des films
    all_movies = moroccan_movies + top_movies
    db.movies.insert_many(all_movies)

    # Utilisateurs 
    users_data = []
    for i in range(1, 10):
        num_movies = random.randint(2, 5)  # 2 to 5 movies watched per user
        watched_movies = random.sample(all_movies, num_movies)
        history = [
            {
                "movie_id": movie["_id"],
                "rating": random.randint(1, 5),  # Random rating between 1 and 5
                "watched_on": (datetime.now() - timedelta(days=random.randint(1, 100))).isoformat()
            }
            for movie in watched_movies
        ]
        users_data.append({
            "_id": f"user_{str(i).zfill(3)}",
            "username": f"user{i}",
            "email": f"user{i}@example.com",
            "history": history
        })

    db.users.insert_many(users_data)

    print("[seed_data] Données de test insérées avec succès, y compris l'historique utilisateur !")



In [25]:

# ----------------------------------------------------
# 3) Construction de la matrice utilisateur-film
# ----------------------------------------------------

In [8]:

def build_user_movie_matrix():
    """
    Construit une matrice utilisateur-film (DataFrame) 
    à partir de la collection 'users'.
    - Lignes : user_id
    - Colonnes : movie_id
    - Valeurs : rating (défaut = 0 si non noté)
    """
    users_docs = list(db.users.find({}))
    data_rows = []
    for user_doc in users_docs:
        user_id = user_doc["_id"]
        for h in user_doc.get("history", []):
            data_rows.append([user_id, h["movie_id"], h["rating"]])
    
    if not data_rows:
        # Si aucune note
        return pd.DataFrame()

    df = pd.DataFrame(data_rows, columns=["user_id", "movie_id", "rating"])
    pivot_df = df.pivot_table(
        index="user_id",
        columns="movie_id",
        values="rating",
        fill_value=0
    ) 
    return pivot_df



In [26]:
# ----------------------------------------------------
# 4) Calcul de la similarité Cosine
# ----------------------------------------------------

In [9]:

def compute_user_similarity(matrix):
    """
    matrix: DataFrame (user x movie) avec les notes.
    Retourne un DataFrame de similarité (user x user).
    """
    sim = cosine_similarity(matrix)
    sim_df = pd.DataFrame(sim, index=matrix.index, columns=matrix.index)
    return sim_df


In [ ]:

# ----------------------------------------------------
# 5) Fonctions de recommandation (Filtrage Collaboratif)
# ----------------------------------------------------

In [18]:

def collaborative_filtering(user_id, top_n=5):
    """
    Filtrage collaboratif simple (user-based).
    - Construit la matrice
    - Calcule la similarité cosinus
    - Cherche les utilisateurs les plus proches
    - Récupère les films qu'ils ont bien notés (rating >= 4)
    - Exclut les films déjà vus.
    Retourne une liste d'ID de films recommandés.
    """
    matrix = build_user_movie_matrix()
    if matrix.empty or user_id not in matrix.index:
        return []

    # Calcul de la similarité
    sim_df = compute_user_similarity(matrix)

    # Similarité de l'utilisateur user_id avec tous les autres
    user_sim_series = sim_df.loc[user_id].sort_values(ascending=False)
    
    # Retirer l'utilisateur lui-même
    user_sim_series = user_sim_series.drop(user_id, errors='ignore')
    
    # Prenons par ex. les 3 plus proches
    top_neighbors = user_sim_series.head(3).index

    # Extraire la liste des films déjà vus
    user_ratings = matrix.loc[user_id]
    already_seen = user_ratings[user_ratings > 0].index.tolist()

    recommended_movies = []
    for neighbor in top_neighbors:
        neighbor_ratings = matrix.loc[neighbor]
        # On garde ceux notés >= 4
        neighbor_fav = neighbor_ratings[neighbor_ratings >= 4].index.tolist()
        for m in neighbor_fav:
            if m not in already_seen:
                recommended_movies.append(m)

    # Eviter les doublons
    recommended_movies = list(set(recommended_movies))
    return recommended_movies[:top_n]


In [ ]:

# ----------------------------------------------------
# 6) Fonction pour donner une note a un film
# ----------------------------------------------------

In [19]:
def rate_movie(user_id, movie_id, rating):
    """
    Ajoute/modifie la notation d'un film pour un utilisateur donné.
    """
    db.users.update_one(
        {"_id": user_id},
        {
            "$push": {
                "history": {
                    "movie_id": movie_id,
                    "rating": rating,
                    "watched_on": datetime.utcnow().isoformat()  
                }
            }
        }
    )
    print(f"[rate_movie] {user_id} -> {movie_id} = {rating}")


In [ ]:

# ----------------------------------------------------
# 7) Main
# ----------------------------------------------------

In [31]:
if __name__ == "__main__":

    seed_data()

    print("\n=== Films insérés ===")
    for m in db.movies.find({}):
        print(m)

    print("\n=== Utilisateurs insérés ===")
    for u in db.users.find({}):
        print(u)

    print("\n=== Ajouter une notation pour user_002 (The Godfather : 5) ===")
    rate_movie("user_002", "movie_int_001", 5)
    
    # 5) Calculer la matrice utilisateur-film et l'afficher
    print("\n=== Matrice utilisateur-film (après ajout) ===")
    matrix_uf = build_user_movie_matrix()
    print(matrix_uf)

    # 6) Obtenir des recommandations pour user_001
    print("\n=== Recommandations pour user_001 ===")
    recs_user001 = collaborative_filtering("user_001", top_n=5)
    print("Films recommandés =", recs_user001)

 


[seed_data] Données de test insérées avec succès, y compris l'historique utilisateur !

=== Films insérés ===
{'_id': 'movie_ma_001', 'title': 'Ali Zaoua', 'genres': ['Drame'], 'actors': ['Said Taghmaoui', 'Mounim Kbab'], 'description': 'Un regard poignant sur les enfants des rues à Casablanca.', 'release_date': '2000-10-04'}
{'_id': 'movie_ma_002', 'title': 'Les Chevaux de Dieu', 'genres': ['Drame', 'Thriller'], 'actors': ['Abdelhakim Rachid', 'Abdelilah Rachid'], 'description': "L'histoire de jeunes hommes dans un quartier défavorisé.", 'release_date': '2012-05-23'}
{'_id': 'movie_ma_003', 'title': 'Much Loved', 'genres': ['Drame'], 'actors': ['Loubna Abidar', 'Asmaa Lazrak'], 'description': 'Un portrait réaliste de la vie des travailleuses du sexe au Maroc.', 'release_date': '2015-05-19'}
{'_id': 'movie_ma_004', 'title': 'Casablanca Beats', 'genres': ['Musique', 'Drame'], 'actors': ['Ismail Adouab', 'Nouhaila Arif'], 'description': "Un atelier de hip-hop qui inspire les jeunes à s'e